In [1]:
import numpy as np
import librosa
import soundfile
import os, glob, pickle
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
CLASSES = ['Exclamation', 'Question', 'Statement']
PATH_TRAIN = 'C:\\Users\\Sveta\\PycharmProjects\\Course project\\Russian'
PATH_TEST= 'C:\\Users\\Sveta\\PycharmProjects\\Course project\\Russian\\Test'

In [141]:
def extract_feature(file_name, mfcc, chroma, mel):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    if chroma:
        stft=np.abs(librosa.stft(X))
    result=np.array([])
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    if chroma:
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [142]:
def get_data(path):
    X = []
    y = []
    count = 0
    for int_class in CLASSES:
        class_path = os.path.join(path, int_class)
        #print(int_class)
        for filename in tqdm(os.listdir(class_path)):
            if not '.wav' in filename:
                continue
            filepath = glob.glob(os.path.join(class_path, filename))[0]
            feature_vector = extract_feature(filepath, mfcc=True, chroma=True, mel=True)
            X.append(feature_vector)
            y.append(int_class)
            count += 1
    print('Set from ' + path + ' uploaded. \nFiles ' + str(count))
    return X, y

In [143]:
X_train, y_train = get_data(PATH_TRAIN)

100%|███████████████████████████████████████████████████████████████████████████████████████| 66/66 [00:03<00:00, 17.74it/s]

Set from C:\Users\Sveta\PycharmProjects\Course project\Russian uploaded. 
Files 246


In [144]:
X_test, y_test = get_data(PATH_TEST)

100%|███████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:01<00:00, 17.40it/s]

Set from C:\Users\Sveta\PycharmProjects\Course project\Russian\Test uploaded. 
Files 66


In [145]:
from sklearn.linear_model import RidgeClassifier
from sklearn import metrics

ridge = RidgeClassifier(alpha=0.01, solver="sag")
ridge.fit(X_train, y_train)

RidgeClassifier(alpha=0.01, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None, solver='sag',
                tol=0.001)

In [146]:
y_pred = ridge.predict(X_test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

 Exclamation       0.84      0.80      0.82        20
    Question       0.73      0.83      0.78        23
   Statement       0.95      0.87      0.91        23

    accuracy                           0.83        66
   macro avg       0.84      0.83      0.84        66
weighted avg       0.84      0.83      0.84        66



In [147]:
import pickle
# save the classifier
with open('ridge84.pkl', 'wb') as fid:
    pickle.dump(ridge, fid)    

# load it again
with open('ridge.pkl', 'rb') as fid:
    model = pickle.load(fid)


In [148]:
y_pred2 = model.predict(X_test)
print(metrics.classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

 Exclamation       0.84      0.80      0.82        20
    Question       0.73      0.83      0.78        23
   Statement       0.95      0.87      0.91        23

    accuracy                           0.83        66
   macro avg       0.84      0.83      0.84        66
weighted avg       0.84      0.83      0.84        66

